In [2]:
file_path = "../.data/mix_model/data/all_raretrible.txt"
# file_path = "/home/azhao/projects/ai/train/liubin/mix_model/data/all_raretrible.txt"
data = open(file_path, "r").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:]), "", "", ""] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。', '', '', ''], ['中性', '没有词汇课的优惠券吗?', '', '', '']]


In [3]:
import pandas as pd

# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=["label", "text", "pred_label", "score", "keywords"])

# 保存为 CSV 文件
cvs_file = "../.data/data.csv"
df.to_csv(cvs_file, index=False)

In [ ]:
%pip install dspy 

In [5]:
from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_csv(
    cvs_file,
    fields=("label", "text", "score", "pred_label", "score", "keywords"),
    input_keys=("text",),
)
print(all_dataset[0:2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

e:\dev\anaconda3\envs\rag\Lib\site-packages\datasets\download\streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


[Example({'label': '感激', 'text': '好的,那我谢谢您了。', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys={'text'})]


In [6]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [7]:
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 5, '焦急': 3, '高兴': 3, '抱怨': 3, '中性': 2, '惊讶': 2, '生气': 2})
Counter({'中性': 8, '感激': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '抱怨': 7, '生气': 7})
Counter({'中性': 15, '惊讶': 15, '感激': 14, '焦急': 14, '高兴': 14, '抱怨': 14, '生气': 14})


In [ ]:
for ex in sample_set:
    print(ex.get("label"), ex.get("text"))

## base qa


In [11]:
import os

import dspy
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), override=True)
api_key = os.getenv("ONEAPI_API_KEY")
base_url = os.getenv("ONEAPI_BASE_URL")

# 测试要加 model_type ，不然因 model 不存在，会默认 text
turbo = dspy.OpenAI(
    api_key=api_key,
    model="glm-4-flash",
    api_base=base_url,
    model_type="chat",
    max_tokens=8000,
)

dspy.settings.configure(
    lm=turbo,
)

In [24]:
import dspy
from dsp import passages2text


class BasicQA(dspy.Signature):
    """sentiment analysis from text"""

    question = dspy.InputField(desc="list of input text", format=passages2text)
    answer = dspy.OutputField(
        desc="返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9 ",
        type=list,
    )

In [25]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

samples = [x.text for x in sample_set[0:20]]
# Call the predictor on a particular input.
pred = generate_answer(question=samples)

# Print the input and the prediction.
print(f"Question: {samples}")
print(f"Predicted Answer:\n{pred.answer}")

Question: ['谢谢谢谢拜托了', '好的,那我知道了,谢谢哈', '知就好了,谢谢你!', '好的。尽快谢谢', '谢谢喔', '暂时没有了,有什么需要我再问。谢谢', '好快点啊谢谢你了', '算了,谢谢你了', '好的,明白,谢谢!!', '哦谢谢亲', '谢谢,已经查到了', '好吧!麻烦了谢谢', '哦呼的,好的谢谢你', '好的明白了,谢谢', '订购破人和付款人不一致', '我是跟好友进去的', '请告知客服电话', '我也不明白你的意思', '乙到了终点站了', '请问怎么查询订单?']
Predicted Answer:
1, 正向, 感激, 0.8
2, 正向, 感激, 0.7
3, 正向, 感激, 0.8
4, 正向, 感激, 0.6
5, 正向, 感激, 0.7
6, 正向, 感激, 0.6
7, 正向, 感激, 0.7
8, 正向, 感激, 0.6
9, 正向, 感激, 0.8
10, 正向, 感激, 0.7
11, 正向, 感激, 0.8
12, 正向, 感激, 0.7
13, 正向, 感激, 0.7
14, 正向, 感激, 0.7
15, 中性, 中性, 0.5
16, 中性, 中性, 0.5
17, 中性, 中性, 0.5
18, 中性, 中性, 0.5
19, 中性, 中性, 0.5
20, 中性, 中性, 0.5


In [26]:
# 答案和问题对应起来
def update_sample(sample_set, answer):
    for sample, prediction in zip(sample_set, answer.split("\n")):
        sample["score"] = prediction.split(",")[3]
        sample["pred_label"] = prediction.split(",")[2]
        sample["keywords"] = prediction.split(",")[1]


def sample_error_rate(sample_set):
    incorrect_count = sum(
        1 for sample in sample_set if sample["pred_label"] != sample["label"]
    )
    total_count = len(sample_set)
    return incorrect_count / total_count


update_sample(sample_set, pred.answer)
print(sample_error_rate(sample_set))

for sample in sample_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

1.0
Question: Example({'label': '感激', 'text': '谢谢谢谢拜托了', 'score': ' 0.8', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好的,那我知道了,谢谢哈', 'score': ' 0.7', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '知就好了,谢谢你!', 'score': ' 0.8', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好的。尽快谢谢', 'score': ' 0.6', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '谢谢喔', 'score': ' 0.7', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '暂时没有了,有什么需要我再问。谢谢', 'score': ' 0.6', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好快点啊谢谢你了', 'score': ' 0.7', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '算了,谢谢你了'

In [27]:
turbo.inspect_history(n=1)





sentiment analysis from text

---

Follow the following format.

Question: list of input text
Answer: 返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9

---

Question:
[1] «谢谢谢谢拜托了»
[2] «好的,那我知道了,谢谢哈»
[3] «知就好了,谢谢你!»
[4] «好的。尽快谢谢»
[5] «谢谢喔»
[6] «暂时没有了,有什么需要我再问。谢谢»
[7] «好快点啊谢谢你了»
[8] «算了,谢谢你了»
[9] «好的,明白,谢谢!!»
[10] «哦谢谢亲»
[11] «谢谢,已经查到了»
[12] «好吧!麻烦了谢谢»
[13] «哦呼的,好的谢谢你»
[14] «好的明白了,谢谢»
[15] «订购破人和付款人不一致»
[16] «我是跟好友进去的»
[17] «请告知客服电话»
[18] «我也不明白你的意思»
[19] «乙到了终点站了»
[20] «请问怎么查询订单?»
Answer: 1, 正向, 感激, 0.8
2, 正向, 感激, 0.7
3, 正向, 感激, 0.8
4, 正向, 感激, 0.6
5, 正向, 感激, 0.7
6, 正向, 感激, 0.6
7, 正向, 感激, 0.7
8, 正向, 感激, 0.6
9, 正向, 感激, 0.8
10, 正向, 感激, 0.7
11, 正向, 感激, 0.8
12, 正向, 感激, 0.7
13, 正向, 感激, 0.7
14, 正向, 感激, 0.7
15, 中性, 中性, 0.5
16, 中性, 中性, 0.5
17, 中性, 中性, 0.5
18, 中性, 中性, 0.5
19, 中性, 中性, 0.5
20, 中性, 中性, 0.5





In [28]:
# 加上COT
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_by_cot = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_by_cot(question=samples)

# Print the input, the chain of thought, and the prediction.
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Thought: We will analyze each text individually to determine the sentiment expressed, which can be positive, negative, or neutral. We will then classify the sentiment into one of the following categories: 中性 (neutral), 惊讶 (surprise), 感激 (gratitude), 抱怨 (complaint), 焦急 (anxiety), 生气 (anger), 高兴 (happiness), and assign a score between 0 and 1.0 to represent the strength of the sentiment.
Predicted Answer: 1, 正向, 感激, 0.8
2, 正向, 感激, 0.7
3, 正向, 感激, 0.9
4, 正向, 感激, 0.6
5, 正向, 感激, 0.7
6, 正向, 感激, 0.8
7, 正向, 感激, 0.9
8, 正向, 感激, 0.7
9, 正向, 感激, 0.9
10, 正向, 感激, 0.8
11, 正向, 感激, 0.7
12, 正向, 感激, 0.9
13, 正向, 感激, 0.8
14, 正向, 感激, 0.9
15, 负向, 抱怨, 0.6
16, 中性, 中性, 0.5
17, 中性, 中性, 0.5
18, 负向, 抱怨, 0.7
19, 中性, 中性, 0.5
20, 中性, 中性, 0.5


In [29]:
update_sample(sample_set, pred.answer)
print(sample_error_rate(sample_set))

for sample in sample_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

1.0
Question: Example({'label': '感激', 'text': '谢谢谢谢拜托了', 'score': ' 0.8', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好的,那我知道了,谢谢哈', 'score': ' 0.7', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '知就好了,谢谢你!', 'score': ' 0.9', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好的。尽快谢谢', 'score': ' 0.6', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '谢谢喔', 'score': ' 0.7', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '暂时没有了,有什么需要我再问。谢谢', 'score': ' 0.8', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '好快点啊谢谢你了', 'score': ' 0.9', 'pred_label': ' 感激', 'keywords': ' 正向'}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '算了,谢谢你了'

In [30]:
turbo.inspect_history(n=1)





sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9

---

Question:
[1] «谢谢谢谢拜托了»
[2] «好的,那我知道了,谢谢哈»
[3] «知就好了,谢谢你!»
[4] «好的。尽快谢谢»
[5] «谢谢喔»
[6] «暂时没有了,有什么需要我再问。谢谢»
[7] «好快点啊谢谢你了»
[8] «算了,谢谢你了»
[9] «好的,明白,谢谢!!»
[10] «哦谢谢亲»
[11] «谢谢,已经查到了»
[12] «好吧!麻烦了谢谢»
[13] «哦呼的,好的谢谢你»
[14] «好的明白了,谢谢»
[15] «订购破人和付款人不一致»
[16] «我是跟好友进去的»
[17] «请告知客服电话»
[18] «我也不明白你的意思»
[19] «乙到了终点站了»
[20] «请问怎么查询订单?»
Reasoning: Let's think step by step in order to produce the answer. We will analyze each text individually to determine the sentiment expressed, which can be positive, negative, or neutral. We will then classify the sentiment into one of the following categories: 中性 (neutral), 惊讶 (surprise), 感激 (gratitude), 抱怨 (complaint), 焦急 (anxiety), 生气 (anger), 高兴 (happiness), and as

In [33]:
from dspy.teleprompt import BootstrapFewShot

import dspy
from dsp import passages2text

class BasicQA(dspy.Signature):
    """sentiment analysis from text"""

    question = dspy.InputField(desc="list of input text", format=passages2text)
    answer = dspy.OutputField(
        desc="返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9 ",
        type=list,
    )


class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(BasicQA)

    # 这里的 forward 会被调用 Sample 的 inputs_keys 作为参数,
    # 比如那里的 text 在这里，需要作为参数传给 forward(text = sample.get("text"))
    def forward(self, text=None):
        return self.prog(question=text)


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    update_sample([example], pred.answer)
    return example["label"] == example["pred_label"]


# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_cot = teleprompter.compile(CoT(), trainset=train_set)

 25%|██▌       | 5/20 [00:10<00:31,  2.13s/it]

Failed to run or to evaluate example Example({'label': '焦急', 'text': '别人很急', 'score': '0.5 惊讶', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'}) with <function validate_context_and_answer at 0x000001C830E6D800> due to list index out of range.


 45%|████▌     | 9/20 [00:16<00:20,  1.83s/it]

Bootstrapped 4 full traces after 10 examples in round 0.


In [35]:
turbo.inspect_history(n=2)





sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9

---

Question: 我够买的物料,显示五号就停止接单了,这又收到货了,怎么解释
Reasoning: Let's think step by step in order to produce the answer. We need to analyze the sentiment of the text provided. The text expresses confusion and a sense of disbelief about a situation where the speaker expected a service to stop accepting orders but received goods instead. This indicates a negative sentiment.

Answer: 1,负向,抱怨,0.8







sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9

---

Question: 哈哈哈哈可以的亲

In [36]:
text = "\n".join(samples)

samples2 = [x.text for x in test_set]
text = "\n".join(samples2)

pred = compiled_cot(text=text)

In [37]:
turbo.inspect_history(n=1)





sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 返回格式 序号,正向|负向|中性,情绪分类,情绪分类评分 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，情绪分类评分(0-1.0)。例如 1,负向,抱怨,0.9

---

Question: 好的谢谢!!
Reasoning: Let's think step by step in order to produce the answer. We need to analyze the sentiment of the given text "好的谢谢!!". The text is positive and shows gratitude. The word "好的" means "good" or "ok," which is a neutral or positive expression, and "谢谢" means "thank you," which is clearly positive and shows gratitude.
Answer: 1,正向,感激,0.8

---

Question: 谢了亲电话打过了
Reasoning: Let's think step by step in order to produce the answer. We need to analyze the sentiment of the text "谢谢亲电话打过了". The text is a casual and friendly response, which indicates a positive sentiment. The words "谢谢" (thank you) and "亲" (a term of endearment) suggest a sense of gratitude or appreciation. Ther

In [38]:
print(pred.answer)

1,正向,感激,0.8
2,正向,感激,0.8
3,正向,感激,0.8
4,正向,感激,0.8
5,正向,感激,0.8
6,正向,感激,0.8
7,正向,感激,0.8
8,正向,感激,0.8
9,负向,抱怨,0.7
10,负向,抱怨,0.7
11,中性,询问,0.5
12,中性,询问,0.5
13,负向,抱怨,0.7
14,负向,抱怨,0.7
15,中性,紧急,0.6
16,负向,抱怨,0.7
17,正向,感激,0.8
18,正向,感激,0.8
19,中性,紧急,0.6
20,正向,感激,0.8
21,正向,感激,0.8
22,负向,抱怨,0.7
23,负向,抱怨,0.7
24,中性,询问,0.5
25,负向,抱怨,0.7
26,负向,抱怨,0.7
27,负向,抱怨,0.7
28,负向,抱怨,0.7
29,中性,赞赏,0.6
30,正向,满意,0.8
31,正向,满意,0.8
32,正向,满意,0.8
33,中性,赞赏,0.6
34,正向,满意,0.8
35,正向,满意,0.8
36,负向,抱怨,0.7
37,负向,抱怨,0.7
38,负向,抱怨,0.7
39,负向,抱怨,0.7
40,负向,抱怨,0.7
41,负向,抱怨,0.7
42,负向,抱怨,0.7
43,负向,抱怨,0.7
44,负向,抱怨,0.7
45,负向,抱怨,0.7
46,负向,抱怨,0.7
47,负向,抱怨,0.7
48,负向,抱怨,0.7
49,负向,抱怨,0.7
50,负向,抱怨,0.7


In [39]:
update_sample(test_set, pred.answer)
print(sample_error_rate(test_set))

for sample in test_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

0.72
Question: Example({'label': '中性', 'text': '亲,帮忙解封一下旺旺,谢谢', 'score': '0.8', 'pred_label': '感激', 'keywords': '正向'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '点的时候乱跳', 'score': '0.7', 'pred_label': '抱怨', 'keywords': '负向'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '你给送的也不是充值钱数的积分啊', 'score': '0.7', 'pred_label': '抱怨', 'keywords': '负向'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '我这个还需要写售后订单编号?', 'score': '0.5', 'pred_label': '询问', 'keywords': '中性'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '账单期数还有多少期', 'score': '0.5', 'pred_label': '询问', 'keywords': '中性'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '**这个店借的充电宝不让我在这个店还?', 'score': '0.7', 'pred_label': '抱怨', 'keywords': '负向'}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '那没有准考证呀', 'score': '0.7', 'pred_label': '抱怨', 'keywords': '负向'}) (input_keys={'text'})
Question: Example({'label': '焦急', 'text': '这个很紧急', 